# Power Spectrum Analysis

## Configuration

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import pickle
import numpy as np
import healpy as hp
import matplotlib.pyplot as plt
import matplotlib as mpl
from deep_dss.helpers import *
import pymaster as nmt
import warnings

warnings.filterwarnings(action='once')

# Run on GPU.
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [3]:
# Config variables
config = "k"
zscale = True
nside = 1024
order = 2
gaussian = False
free_bias = False
prior_low = 0.8
prior_high = 3.0

/Users/adiraju/opt/anaconda3/envs/deep_dss/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [5]:
# Using covariance Q1 data
data = split_count_and_lensing_maps_by_dataset("COVQ1", config=config, covariance=True, zscale=zscale)

/Users/adiraju/opt/anaconda3/envs/deep_dss/lib/python3.8/site-packages/healpy/fitsfunc.py:368: UserWarning: If you are not specifying the input dtype and using the default np.float64 dtype of read_map(), please consider that it will change in a future version to None as to keep the same dtype of the input file: please explicitly set the dtype if it is important to you.
  warnings.warn(


FileNotFoundError: [Errno 2] No such file or directory: '../data/flaskv4/output/dss-201.0/dss-201.0-map-f1z1.fits.gz'